In [1]:
"""
This script was used to debug the behaviour data. It was used to find the errors in the behaviour data and to find the errors in the code that extracts the timestamps and events from the raw behaviour data.

Normally this code is not needed anymore...
"""

#%%
import sys
sys.path.append("C:\ZSOMBI\OneDrive\PoPy")

import pandas as pd
import numpy as np
import scipy.io as sio

from popy.io_tools import load_raw_behav_mat
from popy.io_tools import get_behavior

def is_file_valid(behav):
    """
    Tries to find errors in the behaviour data. Returns True if no errors were found, False otherwise. However, it is not guaranteed that the function will find all errors.
    """
    locs_trial_start = np.argwhere(behav[:, 0] == 100)
    locs_trial_end = np.argwhere(behav[:, 0] == 101)

    # trial starts NOT equal to trial ends
    if len(locs_trial_start) != len(locs_trial_end):
        print('Num trial start events s not equal to num trial end events')
        if locs_trial_start[0][0] > locs_trial_end[0][0]:
            print('\tperhaps missing first trial start')
        if locs_trial_end[-1][0] < locs_trial_start[-1][0]:
            print('\tperhaps missing last trial end')

        # trial ends should be followed by trial starts, except when it if a block end (in this case there are 2 excess values in between)
        for start_id in locs_trial_start[1:]:  # ignore first one as it has no previous
            start_id = start_id[0]
            if not (behav[start_id - 1, 0] == 101 or behav[start_id - 3, 0] == 101):
                print('\tmissing trial end at: ', start_id)
        for end_id in locs_trial_end[:-1] :  # ignore last one as it has no following
            end_id = end_id[0]
            if not (behav[end_id + 1, 0] == 100 or behav[end_id + 3, 0] == 100):
                print('\tmissing trial start at: ', end_id)

        return False

    for start_id, end_id in zip(locs_trial_start, locs_trial_end):
        start_id, end_id = start_id[0], end_id[0]
        # behav of current trial
        behav_curr = behav[start_id:end_id+1, 0]

        # num events
        all_events = {
            'N_trial_start': np.count_nonzero(behav_curr == 100),
            'N_lever_val': np.count_nonzero(behav_curr == 62),
            'N_lever_release': np.count_nonzero(behav_curr == 64),
            'N_target_valid': np.count_nonzero(behav_curr== 125),
            'N_fb': np.count_nonzero((behav_curr == 65) | (behav_curr == 66)),
            'N_trial_end': np.count_nonzero(behav_curr == 101),
            'N_no_touch': np.count_nonzero(behav_curr == 252)
        }

        # is there a value in all_events higher than 1?
        if np.any(np.array(list(all_events.values())) > 1):
            multiples = np.argwhere(np.array(list(all_events.values())) > 1)
            print(f'corrupt trial between index {start_id} and {end_id}')
            print(f'\tmultiple behav code for {np.array(list(all_events.keys()))[multiples]}')
            return False

        # when there is not a 'no touch' event, i.e. correct trial
        if all_events['N_no_touch'] == 0:
            N_events = np.array(list(all_events.values()))[:-1] # all events - remove last element (N_no_touch)
            if all([x == 1 for x in N_events]):  # all events must be 1, in this case its a correct trial
                continue 
            else:
                # is there any value of 0?
                zeros = np.argwhere(N_events == 0)
                print(f'corrupt trial between index {start_id} and {end_id}')
                print(f'\tmissing behav code for {np.array(list(all_events.keys()))[zeros]}')
                return False
        else:  # in case there is a 'no touch' event, i.e. incorrect trial
            N_events = np.array(list(all_events.values()))[:-2]  # all events - remove last 2 elements (N_no_touch, N_trial_end)
            # is the ordering right?
            ordering_right = [x >= y for x, y in zip(N_events[:-1], N_events[1:])]
            if all(ordering_right):
                continue
            else:
                print(f'corrupt trial between index {start_id} and {end_id}')
                print('there is a no touch event, but something is missing from before')
                return False
            
    # if it didnt find an error, return True
    return True

Using OFFICE path
Inserm drive is not accessible


# Check if file is valid

Processing po 261020
Warning: no best target of multiple best targets in block (0).

In [2]:
monkey, session = 'ka', '310522'

rec = load_raw_behav_mat(monkey, session, 'C:\\ZSOMBI\\OneDrive\\PoPy\\data\\recordings\\behavior')

# locate errors
if is_file_valid(rec):
    print('file is valid')

file is valid


In [3]:
rec

array([[1.00000000e+02, 6.30830000e+01],
       [5.10000000e+01, 6.30836333e+01],
       [1.00000000e+00, 6.31230000e+01],
       ...,
       [1.44000000e+02, 2.01292477e+03],
       [1.44000000e+02, 2.01292717e+03],
       [1.01000000e+02, 2.01338057e+03]])

# Check if file can be converted without problems

In [4]:
session_data = get_behavior(monkey, session)  # load raw data and preprocess it
# show full dataframe
pd.set_option('display.max_rows', None)
session_data

ValueError: Trial id 232 not found in any block.

# Correct mistakes

In [ ]:
# get corresponding filename (to load raw data)
all_session_info = pd.read_csv('C:\\ZSOMBI\\OneDrive\\PoPy\\data\\recordings_summary.csv', dtype={'monkey': str, 'session': str})
fname = all_session_info.loc[(all_session_info['monkey'] == monkey) & (all_session_info['session'] == session), 'behav_file_name'].values[0]
print(fname)

ka310522_fpkk_4785003.mat


In [ ]:
# load mat file
mat = sio.loadmat(f'C:\\ZSOMBI\\OneDrive\\PoPy\\data\\recordings\\behavior\\{fname}')
# save mat file
behav = mat['behav']

#%% Correct errors
#mat['behav'] = behav[:2414, :]
#mat['behav'][4659, 0] = 61
#mat['behav'] = np.delete(mat['behav'], 6900, 0)
#t = (mat['behav'][3958, 1] + mat['behav'][39, 1])/2
#mat['behav'] = np.insert(mat['behav'], 3180, [100, t], axis=0)

#behav_new = mat['behav']

#sio.savemat(f'C:\\ZSOMBI\\OneDrive\\PoPy\\data\\recordings\\{fname}', mat)
#mat['behav'] = np.delete(mat['behav'], 5184, 0)
mat['behav'] = np.insert(mat['behav'], 3396, [252, 2342.4002], axis=0)


#mat['behav'][7052, :]
"""
# remove index 814 and 815
mat['behav'] = np.delete(mat['behav'], 814, 0)
mat['behav'] = np.delete(mat['behav'], 814, 0)
t = mat['behav'][814, 1]
mat['behav'] = np.insert(mat['behav'], 815, [7, t+.001], axis=0)
mat['behav'] = np.insert(mat['behav'], 816, [51, t+.002], axis=0)"""

#sio.savemat(f'C:\\ZSOMBI\\OneDrive\\PoPy\\data\\recordings\\behavior\\{fname}', mat)

"\n# remove index 814 and 815\nmat['behav'] = np.delete(mat['behav'], 814, 0)\nmat['behav'] = np.delete(mat['behav'], 814, 0)\nt = mat['behav'][814, 1]\nmat['behav'] = np.insert(mat['behav'], 815, [7, t+.001], axis=0)\nmat['behav'] = np.insert(mat['behav'], 816, [51, t+.002], axis=0)"